In [99]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder,StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,classification_report
import tensorflow as tf
from scikeras.wrappers import KerasClassifier

In [100]:
train_data=pd.read_csv("../dataset/train_data.csv")
test_data=pd.read_csv("../dataset/test_data.csv")

In [101]:
print(f"Shape of the Training data:{train_data.shape}")
print(f"Shape of the Testing data:{test_data.shape}")

Shape of the Training data:(1550, 13)
Shape of the Testing data:(388, 13)


In [102]:
train_data=train_data=train_data.drop(["Unnamed: 0"],axis=1)
test_data=test_data.drop(["Unnamed: 0"],axis=1)

In [103]:
##split train data into the X_train and Y_train
X_train=train_data.drop(["delivery_status"],axis=1)
Y_train=train_data["delivery_status"]

In [104]:
##Adjust the datatype of  assembly_service_requested column
X_train["assembly_service_requested"]=X_train["assembly_service_requested"].astype("object")

In [105]:
# ### Numbercal columns 
numerical_features=X_train.select_dtypes(include="number").columns.to_list()
categorical_features=X_train.select_dtypes(include="object").columns.to_list()

###Nominal variables
categorical_features.remove("brand")
##Ordinal variables
ordinal_features=["brand"]


In [106]:
### Define pipelines
nominal_pipeline=Pipeline(steps=[
    ("One-Hot-Encoder",OneHotEncoder(sparse_output=False,handle_unknown="ignore"))
])
numerical_pipeline=Pipeline(steps=[
    ("Standard Scaler",StandardScaler())
])

ordinal_pipeline=Pipeline(steps=[
    ("Ordinal-Encoder",OrdinalEncoder())
])


transfomer=ColumnTransformer(transformers=[
    ("Numerical Pipeline",numerical_pipeline,numerical_features),
    ("Nominal Pipeline",nominal_pipeline,categorical_features),
    ("Ordinal Pipeline",ordinal_pipeline,ordinal_features)
])

final_pipeline=Pipeline(steps=[
    ("Transfomer",transfomer)
  
])

In [107]:
X_train_preprocessed = final_pipeline.named_steps["Transfomer"].fit_transform(X_train)

# Get the ColumnTransformer from the pipeline
preprocessor = final_pipeline.named_steps["Transfomer"]

# Get feature names from the ColumnTransformer
feature_names = preprocessor.get_feature_names_out()
X_train_preprocessed=pd.DataFrame(X_train_preprocessed,columns=feature_names)

In [108]:
##split train data into the X_train and Y_train
X_test=test_data.drop(["delivery_status"],axis=1)
Y_test=test_data["delivery_status"]

##Adjust the datatype of  assembly_service_requested column
X_test["assembly_service_requested"]=X_test["assembly_service_requested"].astype("object")

X_test_preprocessed = final_pipeline.named_steps["Transfomer"].transform(X_test)
X_test_preprocessed=pd.DataFrame(X_test_preprocessed,columns=feature_names)

In [109]:
##mapping y variable with the numeric values
y_map={"Failed Delivery":0,"On Going":1,"Delivered":2}
Y_train=Y_train.map(y_map)
Y_test=Y_test.map(y_map)

In [110]:
##implementing ANN using tensorflow
model=tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_preprocessed.shape[1],)), ##Input Layer
    tf.keras.layers.Dense(10,activation="relu"), ## First Hidden Layer
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(6,activation="relu"), ##Second Hidden Layer
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(8,activation="relu"), ##Third Hidden Layer
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(3,activation="softmax") ##Outpur Layer
])

model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.fit(X_train_preprocessed,Y_train,validation_data=(X_test_preprocessed,Y_test),epochs=200,batch_size=20,verbose=1)

Epoch 1/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.3774 - loss: 1.4833 - val_accuracy: 0.3969 - val_loss: 1.1347
Epoch 2/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4013 - loss: 1.3711 - val_accuracy: 0.4820 - val_loss: 1.0581
Epoch 3/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4174 - loss: 1.2325 - val_accuracy: 0.4897 - val_loss: 1.0395
Epoch 4/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4394 - loss: 1.1575 - val_accuracy: 0.4897 - val_loss: 1.0344
Epoch 5/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4316 - loss: 1.1592 - val_accuracy: 0.4923 - val_loss: 1.0322
Epoch 6/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4387 - loss: 1.1280 - val_accuracy: 0.4923 - val_loss: 1.0300
Epoch 7/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4548 - loss: 1.0873 - val_accuracy: 0.4948 - val_loss: 1.0288
Epoch 8/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4626 - loss: 1.0592 - val_accuracy: 0.4948 - v

In [111]:
y_pred=model.predict(X_test_preprocessed)
y_pred

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


array([[0.48995084, 0.32754368, 0.18250549],
       [0.48729074, 0.32639098, 0.1863183 ],
       [0.48826674, 0.32774466, 0.18398859],
       ...,
       [0.48755074, 0.32700416, 0.18544509],
       [0.4874588 , 0.32678667, 0.18575458],
       [0.48730198, 0.32641733, 0.18628076]],
      shape=(388, 3), dtype=float32)

In [112]:
y_pred= np.argmax(y_pred, axis=1)
accuracy_score(y_pred,Y_test)

0.4948453608247423

In [113]:
y_pred_train=model.predict(X_train_preprocessed)
y_pred_train= np.argmax(y_pred_train, axis=1)
accuracy_score(y_pred_train,Y_train)

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


0.49483870967741933